In [ ]:
import pandas, os, sys, re, collections, importlib, glob
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Union, Mapping

In [ ]:

def build_96well_transfection_amounts_from_table(
    protein_order: List[str] , concentrations: List[float], n_reps: int = 3,
    ng_to_master_mix: int = 200) -> List[List[List[str]]]:
    
    transformation_amounts = [ng_to_master_mix/x for x in concentrations]
    text_values = ['{}\n{:.3}'.format(x, y) for (x, y) in zip(protein_order, transformation_amounts)]
    return build_96well_array_from_protein_order(text_values=text_values, n_reps=n_reps)

def fill_plate_with_values(text_values: List[str]) -> List[List[str]]:
    cols = [x for x in range(0, 12)]
    rows = 'ABCDEFGH'
    table = []
    current_row = -1
    done = not bool(len(text_values))
    
    while not done:
        for row in rows:
            current_row += 1
            if current_row < 8:
                table.append(['' for x in cols])
            else:
                done = True
                break
                
            # Add in text values if any remain.
            for n_col in cols:
                if len(text_values):
                    element = text_values.pop(0)
                    table[current_row][n_col] = element
    return table

def build_96well_array_from_protein_order(
    text_values: List[str], n_reps: int = 3) -> List[List[List[str]]]:

    # Multiply by rep number
    _protein_order = []
    for entry in text_values:
        for rep in range(0, n_reps):
            _protein_order.append(entry)
    print(f"{len(text_values)} constructs times {n_reps} reps is {len(_protein_order)} wells.")

    # If more than 96 wells, separate into separate plates.
    blocks = []
    if len(_protein_order) <= 96:
        blocks.append(_protein_order)
    else:
        while len(_protein_order) > 96:
            blocks.append(_protein_order[:96])
            _protein_order = _protein_order[96:]
        blocks.append(_protein_order[:min(96, len(_protein_order))])

    # Build plates as table of strings.
    print(f"Separated into {len(blocks)} plates.")
    plates = []
    for block in blocks:
        print("A plate:")
        plate = fill_plate_with_values(block)
        for row in plate:
            print(row)
        plates.append(plate)

    return plates


def dataframe_to_plates(df: pandas.DataFrame, n_reps: int = 3, ng_to_master_mix: int = 200) -> List[List[List[str]]]:
    protein_order = df['Protein'].tolist()
    concentrations = [x*1000 for x in df['µg/µL'].tolist()]  # Convert to ng/µL.

    blocks = build_96well_transfection_amounts_from_table(
        protein_order=protein_order, concentrations=concentrations, n_reps=n_reps, ng_to_master_mix=ng_to_master_mix)
    
    return blocks

def plot_plates(blocks: List[List[List[str]]], prefix: str = '') -> None:
    for n_block, block in enumerate(blocks):
        fig, ax = plt.subplots()
        print(block)
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')


        df = pandas.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
        #print(df.values)
        ax.table(
            cellText=block,
            colLabels=[str(x+1) for x in range(len(block[0]))],
            rowLabels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
            loc='center')

        fig.tight_layout()
        plt.savefig(f'figs/plate_{prefix}_{n_block}.ps')
        plt.show()
        plt.clf()
        plt.close()
df = pandas.read_excel('Sequencing_schemes_and_results.xlsx', sheet_name='Transfections')
col = 'STABILITY TRANSF'
df = df.loc[[not bool(pandas.isna(_)) for _ in df[col]], :]
df.sort_values(by=[col], ascending=True, inplace=True)

ng_per_transf = 35
transf_per_mm = 20
ng_to_master_mix = ng_per_transf * transf_per_mm

n_reps = 3

pabpc_control = df.loc[[bool(x=='PABPC4L') for x in df['Control']]]
sf3b1_control = df.loc[[bool(x=='SF3B1') for x in df['Control']]]

plates = dataframe_to_plates(pabpc_control, n_reps=n_reps, ng_to_master_mix=ng_to_master_mix)
plot_plates(plates, prefix='pabpc4l_controlled')
plates = dataframe_to_plates(sf3b1_control, n_reps=n_reps, ng_to_master_mix=ng_to_master_mix)
plot_plates(plates, prefix='sf3b1_controlled')
